# init
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de

## Tasks
- get OpenStreetMap data using the osmnx functionalities in the Overpass API
- clip it to an AOI

## Prerequisites
- have osmnx and geopandas installed
- have an AOI saved as a vector (shapefile or geopackage)

In [ ]:
import osmnx as ox
import geopandas as gpd
from pathlib import Path

# imports

In [ ]:
#import the AOI
AOI_path = Path('../data/processed/')
AOI_file = 'MAN_LIM_BoundingBox_AOI_A.shp'
AOI_gdf = gpd.read_file(AOI_path / AOI_file)

In [ ]:
#bounds provides minx, miny, maxx, maxy
aoi_coords = AOI_gdf.to_crs(epsg='4326').envelope
bbox_df = gpd.GeoDataFrame(geometry=aoi_coords)
print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])

In [ ]:
#get road features from OSM
# List key-value pairs for tags
# Wiki url https://wiki.openstreetmap.org/wiki/Key:highway
tags = {'highway': ['primary', 'secondary', 'tertiary', 'residential', 'unclassified', 'trunk', 'motorway']}
col_list = ['osmid','name','source','geometry', 'highway','surface','lanes','incline'] #OSM has hundreds of cols, we just want a few. For info, check roads.columns

roads = ox.features_from_bbox(north=aoi_coords.bounds.values[0][3],
                              south=aoi_coords.bounds.values[0][1],
                              west=aoi_coords.bounds.values[0][0],
                              east=aoi_coords.bounds.values[0][2],
                              tags=tags)
roads.reset_index(inplace=True)

#filtering out the wrong geometries (POLYGON or POINT) and element types (NODE)
roads = roads.loc[(roads.geometry.type=='LineString')&(roads.element_type == 'way'), col_list]
#convert the CRS of roads to the same as AOI
roads.to_crs(AOI_gdf.crs,inplace=True)

#plot them
roads.plot('highway',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', linewidth=1, zorder=2)

In [ ]:
# get points with place names from OSM
# List key-value pairs for tags
tags = {'place': ['city', 'town', 'village', 'hamlet','neighbourhood']} # see https://wiki.openstreetmap.org/wiki/Key:place for more info
col_list = ['osmid','name','geometry', 'place','population'] #OSM has hundreds of cols, we just want a few. For info, check roads.columns'source',

places = ox.features_from_bbox(north=aoi_coords.bounds.values[0][3],
                              south=aoi_coords.bounds.values[0][1],
                              west=aoi_coords.bounds.values[0][0],
                              east=aoi_coords.bounds.values[0][2],
                              tags=tags)
places.reset_index(inplace=True)

#filtering out the wrong geometries (POLYGON or POINT) and element types (NODE)
places = places.loc[(places.geometry.type=='Point'), col_list] #&(places.element_type == 'way')
#convert the CRS of roads to the same as AOI
places.to_crs(AOI_gdf.crs,inplace=True)

#plot them
places.plot('place',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired',  zorder=2) #linewidth=1,

In [ ]:
#clip using geopandas, the mask is the AOI we imported above
clipped_gdf = gpd.clip(roads, mask=AOI_gdf, keep_geom_type=True)
clipped_gdf.plot('highway',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', linewidth=1, zorder=2)

# export

In [ ]:
#export roads  to a shapefile
clipped_gdf.to_file(AOI_path / 'MAN_TRA_roads_OSM_2024_L.shp')

In [ ]:
#export places to a shapefile
places.to_file(AOI_path / 'MAN_LOC_places_OSM_2024_P.shp')